In [10]:
from langgraph.graph import START, StateGraph, END
from typing import TypedDict, Annotated
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import operator

In [ ]:
load_dotenv()

In [ ]:
class CSSEssayState(TypedDict):
    essay: str
    analysis_feedback: str
    language_feedback: str
    clarity_of_thought_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [16]:
llm = ChatAnthropic(model_name='claude-3-5-sonnet-20241022')

In [17]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description='feedback for the essay')
    score: int = Field(description='Score out of 10 for the essay', ge=0, lt=10)

In [18]:
llm_with_structured_output = llm.with_structured_output(EvaluationSchema)

In [19]:
llm_with_structured_output

RunnableBinding(bound=ChatAnthropic(model='claude-3-5-sonnet-20241022', anthropic_api_url='https://api.anthropic.com', anthropic_api_key=SecretStr('**********'), model_kwargs={}), kwargs={'tools': [{'name': 'EvaluationSchema', 'input_schema': {'properties': {'feedback': {'description': 'feedback for the essay', 'type': 'string'}, 'score': {'description': 'Score out of 10 for the essay', 'exclusiveMaximum': 10, 'minimum': 0, 'type': 'integer'}}, 'required': ['feedback', 'score'], 'type': 'object'}, 'description': ''}], 'ls_structured_output_format': {'kwargs': {'method': 'function_calling'}, 'schema': {'name': 'EvaluationSchema', 'input_schema': {'properties': {'feedback': {'description': 'feedback for the essay', 'type': 'string'}, 'score': {'description': 'Score out of 10 for the essay', 'exclusiveMaximum': 10, 'minimum': 0, 'type': 'integer'}}, 'required': ['feedback', 'score'], 'type': 'object'}, 'description': ''}}, 'tool_choice': {'type': 'tool', 'name': 'EvaluationSchema'}}, conf

In [21]:
def evaluate_analysis(state: CSSEssayState) -> CSSEssayState:
    essay = state['essay']

    prompt = f'Analyse the essay deeply according to the given topic, provide a detailed feedback and also assign a score out of 10 \n {essay}'
    analysis_result = llm_with_structured_output(prompt)

    return {'analysis_feedback': analysis_result.feedback, 'individual_scores': [analysis_result.score]}


def evaluate_language(state: CSSEssayState) -> CSSEssayState:
    essay = state['essay']

    prompt = f'Analyse the language of the essay, provide a detailed feedback and also assign a score from 10 on the basis of the language \n {essay}'
    language_result = llm_with_structured_output(prompt)

    return {'language_feedback': language_result.feedback, 'individual_scores': language_result.score}


def evaluate_thought(state: CSSEssayState) -> CSSEssayState:
    essay = state['essay']

    prompt = f'Analyse the thought level of the person from the essay, provide a detailed feedback and also assign a score out of 10 of the essay \n {essay}'
    thought_result = llm_with_structured_output(prompt)

    return {'clarity_of_thought_feedback': thought_result.feedback, 'individual_scores': thought_result.score}

In [25]:
def final_evaluation(state: CSSEssayState) -> CSSEssayState:
    essay = state['essay']

    prompt = f"Summarize final feedback on the basis of these feedback \n Language Feedback: \n {state['language_feedback']}, \n Thought Feedback: \n {state['clarity_of_thought_feedback']} \n Analysis Feedback \n {state['analysis_feedback']}"

    overall_feedback = llm.invoke(prompt).content

    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [26]:
graph = StateGraph(CSSEssayState)

graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)


graph.add_edge(START, "evaluate_analysis")
graph.add_edge(START, "evaluate_language")
graph.add_edge(START, "evaluate_thought")

graph.add_edge("evaluate_analysis", "final_evaluation")
graph.add_edge("evaluate_language", "final_evaluation")
graph.add_edge("evaluate_thought", "final_evaluation")

graph.add_edge("final_evaluation", END)

workflow = graph.compile()

In [ ]:
essay = """

"""

In [ ]:
initial_state = {'essay': essay}

final_state = workflow.invoke(initial_state)